# **Airfare Price Prediction**

Dennis Myasnyankin, Vannessa Salazar, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 599: Capstone Project

Professor Ebrahim Tarshizi

December 11, 2023

***

## **Data Importing**

### **Import Libraries**

In [29]:
import pandas as pd
from pandas import json_normalize
from datetime import datetime

### **Data Ingestion**

In [2]:
data = pd.read_json('/Users/Chris/Downloads/BCN.json')
data

,groupedItineraryResponse
baggageAllowanceDescs,"[{'id': 1, 'pieceCount': 1}, {'id': 2, 'weight..."
fareComponentDescs,"[{'id': 1, 'governingCarrier': 'VY', 'fareAmou..."
itineraryGroups,[{'groupDescription': {'legDescriptions': [{'d...
legDescs,"[{'id': 1, 'elapsedTime': 130, 'schedules': [{..."
messages,"[{'severity': 'Info', 'type': 'SERVER', 'code'..."
obFeeDescs,"[{'id': 1, 'amount': 0.0, 'currency': 'USD'}]"
scheduleDescs,"[{'id': 1, 'frequency': 'SMTWT**', 'stopCount'..."
statistics,{'itineraryCount': 17}
taxDescs,"[{'id': 1, 'code': 'VV3', 'amount': 28.3, 'cur..."
taxSummaryDescs,"[{'id': 1, 'code': 'YQ', 'amount': 22.5, 'curr..."


In [28]:
schedule_descs = data["groupedItineraryResponse"]["scheduleDescs"]
leg_descs = data["groupedItineraryResponse"]["legDescs"]
itinerary_groups = data["groupedItineraryResponse"]["itineraryGroups"]

# Initialize an empty list to store the extracted data
out = []

# Iterate over itinerary groups
for group in itinerary_groups:
    itineraries = group.get("itineraries", [])
    departure_date = group["groupDescription"]["legDescriptions"][0]["departureDate"]
    departure_city = group["groupDescription"]["legDescriptions"][0]["departureLocation"]
    arrival_date = group["groupDescription"]["legDescriptions"][0].get("arrivalDate", departure_date)
    arrival_location = group["groupDescription"]["legDescriptions"][0]["arrivalLocation"]
    
    # Calculate "Days Until Departure" based on current date and departure date
    current_date = datetime.now()
    departure_date = datetime.strptime(departure_date, "%Y-%m-%d")
    days_until_departure = (departure_date - current_date).days
    
    # Iterate over itineraries in the group
    for itinerary in itineraries:
        legs = itinerary.get("legs", [])
        for leg in legs:
            schedules = leg_descs[leg.get('ref') - 1].get("schedules", [])
            for schedule in schedules:
                schedule_data = schedule_descs[schedule.get('ref') - 1]
                
                departure_time = schedule_data["departure"]["time"].split("+")[0]
                arrival_time = schedule_data["arrival"]["time"].split("+")[0]
                flight_duration = schedule_data["elapsedTime"]
                operating_carrier = schedule_data["carrier"]["operating"]
                aircraft_type = schedule_data["carrier"]["equipment"]["code"]
                cabin_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["cabinCode"]
                fare_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["bookingCode"]
                price = itinerary["pricingInformation"][0]["fare"]["totalFare"]["totalPrice"]
                
                # Add to output
                out.append([
                    departure_city, departure_date, departure_time,
                    arrival_location, arrival_date, arrival_time,
                    flight_duration, operating_carrier, aircraft_type,
                    cabin_class, fare_class, price, days_until_departure
                ])

# Creating a dataFrame
columns = [
    "Departure City", "Departure Date", "Departure Time",
    "Arrival Location", "Arrival Date", "Arrival Time",
    "Flight Duration", "Operating Carrier", "Aircraft Type",
    "Cabin Class", "Fare Class", "Price", "Days Until Departure"
]

df = pd.DataFrame(out, columns = columns)
df

,Departure City,Departure Date,Departure Time,Arrival Location,Arrival Date,Arrival Time,Flight Duration,Operating Carrier,Aircraft Type,Cabin Class,Fare Class,Price,Days Until Departure
0,AMS,2023-11-15,07:00:00,BCN,2023-11-15,09:15:00,135,VY,32A,Y,A,231.4,-6
1,AMS,2023-11-15,07:00:00,BCN,2023-11-15,09:15:00,135,VY,320,Y,V,242.4,-6
2,AMS,2023-11-15,14:55:00,BCN,2023-11-15,17:10:00,135,VY,32A,Y,N,256.4,-6
3,AMS,2023-11-15,10:25:00,BCN,2023-11-15,12:40:00,135,VY,320,Y,F,268.4,-6
4,AMS,2023-11-15,13:30:00,BCN,2023-11-15,15:45:00,135,VY,320,Y,F,268.4,-6
5,AMS,2023-11-15,14:55:00,BCN,2023-11-15,17:10:00,135,VY,320,Y,F,268.4,-6
6,AMS,2023-11-15,10:25:00,BCN,2023-11-15,12:40:00,135,VY,32A,Y,S,269.4,-6
7,AMS,2023-11-15,13:30:00,BCN,2023-11-15,15:45:00,135,VY,32A,Y,S,269.4,-6
8,AMS,2023-11-15,16:30:00,BCN,2023-11-15,18:45:00,135,VY,320,Y,L,281.4,-6
9,AMS,2023-11-15,16:30:00,BCN,2023-11-15,18:45:00,135,VY,32A,Y,V,282.4,-6
